In [2]:
import pandas as pd
import numpy as np
import sklearn
import sklearn.model_selection
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import scipy

In [3]:
def error(true, pred):
    error = np.square(np.subtract(true,pred)).mean(axis=0)
    return error

In [4]:
df = pd.read_excel("raw_data.xlsx",skiprows=1,usecols=["微博正文","点赞数","转发数","评论数"])
df = df.rename(columns = {'微博正文':'text', '点赞数':'like', '转发数':'forward','评论数':'comment'}, inplace=False)

f = open('tfidf.txt', 'r')
allsentences = f.readlines()

for i in range(len(allsentences)):
    allsentences[i] = allsentences[i].strip('\n')
y = np.log(df[['like','forward','comment']])
f.close()
#log几乎不影响

<ipython-input-4-94096a655efd>:9: RuntimeWarning: divide by zero encountered in log
  y = np.log(df[['like','forward','comment']])


In [7]:
t = scipy.sparse.coo_matrix(y)
for i, j, v in zip(t.row, t.col, t.data):
    if (np.isnan(v) or np.isinf(v)):
        y.iloc[i, j] = 0

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(allsentences[:50000], y[:50000], test_size=.3, train_size=.7, random_state=0)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


#该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频
vectorizer = CountVectorizer(max_features = 10000)
#该类会统计每个词语的tf-idf权值
tf_idf_transformer = TfidfTransformer()

tf_idf1 = tf_idf_transformer.fit_transform(vectorizer.fit_transform(X_train))
X_train_weight = tf_idf1
#将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重

#对测试集进行tf-idf权重计算
tf_idf2 = tf_idf_transformer.transform(vectorizer.transform(X_test))
X_test_weight = tf_idf2

In [9]:
svr = sklearn.svm.SVR(kernel ='rbf',degree = 3, gamma ='auto', coef0 = 0.0, tol = 0.001, C = 1.0,
epsilon = 0.1, shrinking = True, cache_size = 200, verbose = False, max_iter = -1)

svr.fit(X_train_weight, y_train['like'])
y_train_pred0 = svr.predict(X_train_weight)
y_test_pred0 = svr.predict(X_test_weight)

svr.fit(X_train_weight, y_train['forward'])
y_train_pred1 = svr.predict(X_train_weight)
y_test_pred1 = svr.predict(X_test_weight)

svr.fit(X_train_weight, y_train['comment'])
y_train_pred2 = svr.predict(X_train_weight)
y_test_pred2 = svr.predict(X_test_weight)

In [10]:
y_train_pred = np.concatenate(([y_train_pred0], [y_train_pred1], [y_train_pred2]), axis=0)
y_test_pred = np.concatenate(([y_test_pred0], [y_test_pred1], [y_test_pred2]), axis=0)

train_error = np.absolute(np.subtract(np.exp(y_train),np.exp(y_train_pred.T))).mean(axis=0)
print(train_error)

test_error = np.absolute(np.subtract(np.exp(y_test),np.exp(y_test_pred.T))).mean(axis=0)
print(test_error)

like       5002.900623
forward    4422.304667
comment    1188.533845
dtype: float64
like       4909.050937
forward    3373.718870
comment    1213.058375
dtype: float64


In [13]:
svr1 = sklearn.svm.SVR(kernel ='poly',degree = 3, gamma ='auto', coef0 = 0.0, tol = 0.001, C = 1.0,
epsilon = 0.1, shrinking = True, cache_size = 200, verbose = False, max_iter = -1)

svr1.fit(X_train_weight, y_train['like'])
y_train_pred0 = svr.predict(X_train_weight)
y_test_pred0 = svr.predict(X_test_weight)

svr1.fit(X_train_weight, y_train['forward'])
y_train_pred1 = svr.predict(X_train_weight)
y_test_pred1 = svr.predict(X_test_weight)

svr1.fit(X_train_weight, y_train['comment'])
y_train_pred2 = svr.predict(X_train_weight)
y_test_pred2 = svr.predict(X_test_weight)

In [14]:
y_train_pred = np.concatenate(([y_train_pred0], [y_train_pred1], [y_train_pred2]), axis=0)
y_test_pred = np.concatenate(([y_test_pred0], [y_test_pred1], [y_test_pred2]), axis=0)
    
train_error = np.absolute(np.subtract(np.exp(y_train),np.exp(y_train_pred.T))).mean(axis=0)
print(train_error)

test_error = np.absolute(np.subtract(np.exp(y_test),np.exp(y_test_pred.T))).mean(axis=0)
print(test_error)

like       5618.077962
forward    4660.494981
comment    1190.669840
dtype: float64
like       5512.656311
forward    3612.471748
comment    1215.079982
dtype: float64


In [15]:
svr2 = sklearn.svm.SVR(kernel ='sigmoid',degree = 3, gamma ='auto', coef0 = 0.0, tol = 0.001, C = 1.0,
epsilon = 0.1, shrinking = True, cache_size = 200, verbose = False, max_iter = -1)

svr2.fit(X_train_weight, y_train['like'])
y_train_pred0 = svr.predict(X_train_weight)
y_test_pred0 = svr.predict(X_test_weight)

svr2.fit(X_train_weight, y_train['forward'])
y_train_pred1 = svr.predict(X_train_weight)
y_test_pred1 = svr.predict(X_test_weight)

svr2.fit(X_train_weight, y_train['comment'])
y_train_pred2 = svr.predict(X_train_weight)
y_test_pred2 = svr.predict(X_test_weight)

KeyboardInterrupt: 

In [ ]:
y_train_pred = np.concatenate(([y_train_pred0], [y_train_pred1], [y_train_pred2]), axis=0)
y_test_pred = np.concatenate(([y_test_pred0], [y_test_pred1], [y_test_pred2]), axis=0)

train_error = np.absolute(np.subtract(np.exp(y_train),np.exp(y_train_pred.T))).mean(axis=0)
print(train_error)

test_error = np.absolute(np.subtract(np.exp(y_test),np.exp(y_test_pred.T))).mean(axis=0)
print(test_error)